In [1]:
import os 
import sys
import subprocess
import pandas as pd
import numpy as np
import pybedtools
from Bio.Seq import Seq
from Bio.SeqUtils import gc_fraction
from gene2probe import *

In the [previous tutorial](https://github.com/Teichlab/gene2probe/blob/main/notebooks/002_make_exon_probes_XIST.ipynb), we walked you through using gene2probe to generate custom probes targeting exons of a human gene.

But what if we are interested in a specific region in that gene, for example to detect a specific isoform? For example, *PTPRC* (CD45) is captured by 2 probes in the VisiumHD probe set. However, none of these probes captures exon 4, which is crucial for distinguishing between CD45RA (containing exon 4; typically expressed in naive and central memory T cells) and CD45RO (missing exon 4; typically expressed in effector memory cells).

Thanks to each modular structure, gene2probe allows us to specify any region(s) of the human genome that are of interest and design probes against them. We can then apply all downstream steps in the same way.

### 1. Specify parameters

In [2]:
## Specify gene of interest and feature of interest
gene_ID = 'PTPRC'
mode = 'exon' ## Whether to consider only exons / introns or full gene

## specify output directory
out_dir = '../sample_run/probeDesign_' + gene_ID + '_' + mode + '/'
## Create output directory
os.makedirs(out_dir, exist_ok=True)

Additionally, we need to provide the path to several resource files. Many of these files can be obtained from [UCSC table browser](https://genome.ucsc.edu/cgi-bin/hgTables).

We also need a blast database, such as the one we generated in the [previous tutorial](https://github.com/Teichlab/gene2probe/blob/main/notebooks/001_make_blast_database.ipynb).

In [3]:
## Required resources (most can be downloaded from 
gtf = '../hg38_resources/hg38.ncbiRefSeq.gtf' ## Gene annotation in gtf file
## We recommend using RefSeq as this is manually curated and more likely to contain an isoform that is present across most cell types
## Alternatively, one can filter based on RNA-seq data for a cell type/tissue of interest
fasta = '../hg38_resources/hg38.fa' ## Genome in fasta file
snp_db = '../hg38_resources/hg38_snp151Common.bed' ## Database of known SNPs and small indels
repeats = '../hg38_resources/hg38_rmsk.bed' ## bed file with repeats/low complexity regions to be excluded
gaps = '../hg38_resources/hg38_rmsk.bed' ## bed file with gaps in the genome assembly to be excluded
blast_db = '../hg38_resources/blastdb/hg38_ncbiRefSeq_transcripts_db' ## Database of all human transcripts to blast against

In [4]:
## Path to blast binaries.
## Replace with your conda environment
## This can also be omitted if you started the jupyter session from within the gene2probe conda environment
blast_exec_path = '/nfs/team205/is10/miniconda/envs/gene2probe_env/bin/'

Finally, we need to provide a set of parameters related to our probe's length, at which nucleotide it's split (if at all), the acceptable range for GC content and any specific requirements for individual nucleotides.

Here we are following the [recommendations of 10x Genomics for custom probes for VisiumHD/VisiumFFPE/Flex](https://cdn.10xgenomics.com/image/upload/v1697739385/support-documents/CG000621_CustomProbeDesign_TechNote_RevC.pdf).

In [42]:
## Additional parameters regarding how the probe should look like
probe_length = 50 ## Length of probe in nucleotides
split_nt = 25 ## Index of nucleotide to split the probe at (start of RHS) - set to None if splitting probe is not needed
min_GC = 0.44 ## Minimum GC content for probe (if split probe, applied to both LHS and RHS)
max_GC = 0.72 ## Maximum GC content for probe (if split probe, applied to both LHS and RHS)
required_nts = {24: 'T'} ## Dictionary of index (0-based) for required nts - by default, 25th nucleotide must be a T - set to None if no requirements
probe_offset = 100 ## Minimum distance between probes - 10 bp is the recommended minimum by 10x, this can also be adjusted depending on how many probes pass other cutoffs
n_desired_probes =1 ## Number of probes to be designed.
min_mismatches = 5 ## Minimum number of mismatches (in at least LHS or RHS) - here we require in both to be more conservative

In [6]:
## Optionally, we can also specify adapters that have to be added to the probes.
## For example, for visiumHD:
LHS_pref = 'CCTTGGCACCCGAGAATTCCA' ## Will be added to the 5' of the LHS probe
LHS_suff = '' ## Will be added to the 3' of the LHS probe
RHS_pref = '/5Phos/' ## Will be added to the 5' of the RHS probe
RHS_suff = 'AAAAAAAAAAAAAAAAAAAAAAAAAAAAAA' ## Will be added to the 3' of the RHS probe

## Leave as empty strings if you don't want to use them

### 2. Generate k-mers

Rather than reading the gene annotation, we can simply define the region of interest:

In [11]:
## Simply replace these coordinates with those of your region of interest.
## You can also add multiple regions, separated by comma
roi_bed = pd.DataFrame(
    {'seqname': ['chr1'],
     'start': [198696712],
     'end': [198696909],
     'name': [(gene_ID + '_0')],
     'score': ['.'],
     'strand': ['+']
    }
)

In [12]:
roi_bed

,seqname,start,end,name,score,strand
0,chr1,198696712,198696909,PTPRC_0,.,+


After extracting the coordinates of interest and converting to a bed-like format, we can generate all possible kmers that fall within these regions.

In [13]:
kmers = generate_kmers(roi_bed, k=probe_length)

In [14]:
kmers

,seqname,start,end,name,score,strand
0,chr1,198696712,198696762,PTPRC_0_0,.,+
1,chr1,198696713,198696763,PTPRC_0_1,.,+
2,chr1,198696714,198696764,PTPRC_0_2,.,+
3,chr1,198696715,198696765,PTPRC_0_3,.,+
4,chr1,198696716,198696766,PTPRC_0_4,.,+
...,...,...,...,...,...,...
143,chr1,198696855,198696905,PTPRC_0_143,.,+
144,chr1,198696856,198696906,PTPRC_0_144,.,+
145,chr1,198696857,198696907,PTPRC_0_145,.,+
146,chr1,198696858,198696908,PTPRC_0_146,.,+


In [15]:
## Export unfiltered
kmers.to_csv((out_dir + 'kmers_all.csv'))

### 3. Exclude annotated repeats/polymorphism

We can next exclude kmers overlapping undesired regions (repeats, low complexity regions, common polymorphism, gaps in the assembly) from further consideration.

In [19]:
## Remove kmers overlapping common polymorphisms
kmers = remove_overlaps(kmers, repeats)

In [20]:
kmers 

,seqname,start,end,name,score,strand
0,chr1,198696712,198696762,PTPRC_0_0,.,+
1,chr1,198696713,198696763,PTPRC_0_1,.,+
2,chr1,198696714,198696764,PTPRC_0_2,.,+
3,chr1,198696715,198696765,PTPRC_0_3,.,+
4,chr1,198696716,198696766,PTPRC_0_4,.,+
...,...,...,...,...,...,...
143,chr1,198696855,198696905,PTPRC_0_143,.,+
144,chr1,198696856,198696906,PTPRC_0_144,.,+
145,chr1,198696857,198696907,PTPRC_0_145,.,+
146,chr1,198696858,198696908,PTPRC_0_146,.,+


In [21]:
## Doing the same for gaps in the assembly (very unlikely since we are starting with annotated exons)
kmers = remove_overlaps(kmers, gaps)

In [22]:
## And more importantly, against common polymorphism (SNPs, short indels)
kmers = remove_overlaps(kmers, snp_db)

In [23]:
kmers

,seqname,start,end,name,score,strand
0,chr1,198696712,198696762,PTPRC_0_0,.,+
1,chr1,198696713,198696763,PTPRC_0_1,.,+
2,chr1,198696714,198696764,PTPRC_0_2,.,+
3,chr1,198696715,198696765,PTPRC_0_3,.,+
4,chr1,198696716,198696766,PTPRC_0_4,.,+
...,...,...,...,...,...,...
143,chr1,198696855,198696905,PTPRC_0_143,.,+
144,chr1,198696856,198696906,PTPRC_0_144,.,+
145,chr1,198696857,198696907,PTPRC_0_145,.,+
146,chr1,198696858,198696908,PTPRC_0_146,.,+


### 4. Filter for desirable sequence features

Having excluded undesirable kmers based on intersection with genomic annotations, the next step is to consider their sequence features.

For this, we first extract the sequences of each k-mer, and then estimate features such as GC content and the presence of desired nucleotides in specific positions.

In [24]:
## Get DNA for the transcript
kmers_bed = pybedtools.BedTool.from_dataframe(kmers)
kmers_seq = kmers_bed.sequence(fi=fasta, s=True) 

## We can read in the sequences and simultaneously monitor GC content and count the longest homopolymer stretch
kmers_seq_stats = get_sequence_stats(kmers_seq.seqfn, probe_length, split_nt)

In [25]:
## Combining with our dataframe
kmers = pd.merge(kmers, kmers_seq_stats, left_index=True, right_index=True)

In [26]:
kmers

,seqname,start,end,name,score,strand,kmer_coord,transcript_seq,probe_seq,GC_content_full,longest_homopolymer,GC_content_LHS,GC_content_RHS
0,chr1,198696712,198696762,PTPRC_0_0,.,+,chr1:198696712-198696762(+),ATTGACTACAGCAAAGATGCCCAGTGTTCCACTTTCAAGTGACCCC...,GTAAGGGGTCACTTGAAAGTGGAACACTGGGCATCTTTGCTGTAGT...,0.46,4,0.48,0.44
1,chr1,198696713,198696763,PTPRC_0_1,.,+,chr1:198696713-198696763(+),TTGACTACAGCAAAGATGCCCAGTGTTCCACTTTCAAGTGACCCCT...,GGTAAGGGGTCACTTGAAAGTGGAACACTGGGCATCTTTGCTGTAG...,0.48,4,0.48,0.48
2,chr1,198696714,198696764,PTPRC_0_2,.,+,chr1:198696714-198696764(+),TGACTACAGCAAAGATGCCCAGTGTTCCACTTTCAAGTGACCCCTT...,AGGTAAGGGGTCACTTGAAAGTGGAACACTGGGCATCTTTGCTGTA...,0.48,4,0.48,0.48
3,chr1,198696715,198696765,PTPRC_0_3,.,+,chr1:198696715-198696765(+),GACTACAGCAAAGATGCCCAGTGTTCCACTTTCAAGTGACCCCTTA...,TAGGTAAGGGGTCACTTGAAAGTGGAACACTGGGCATCTTTGCTGT...,0.48,4,0.48,0.48
4,chr1,198696716,198696766,PTPRC_0_4,.,+,chr1:198696716-198696766(+),ACTACAGCAAAGATGCCCAGTGTTCCACTTTCAAGTGACCCCTTAC...,GTAGGTAAGGGGTCACTTGAAAGTGGAACACTGGGCATCTTTGCTG...,0.48,4,0.48,0.48
...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,chr1,198696855,198696905,PTPRC_0_143,.,+,chr1:198696855-198696905(+),CCACCCAAGTATCCCCGGACTCTTTGGATAATGCTAGTGCTTTTAA...,GGTATTAAAAGCACTAGCATTATCCAAAGAGTCCGGGGATACTTGG...,0.46,4,0.36,0.56
144,chr1,198696856,198696906,PTPRC_0_144,.,+,chr1:198696856-198696906(+),CACCCAAGTATCCCCGGACTCTTTGGATAATGCTAGTGCTTTTAAT...,TGGTATTAAAAGCACTAGCATTATCCAAAGAGTCCGGGGATACTTG...,0.44,4,0.32,0.56
145,chr1,198696857,198696907,PTPRC_0_145,.,+,chr1:198696857-198696907(+),ACCCAAGTATCCCCGGACTCTTTGGATAATGCTAGTGCTTTTAATA...,GTGGTATTAAAAGCACTAGCATTATCCAAAGAGTCCGGGGATACTT...,0.44,4,0.32,0.56
146,chr1,198696858,198696908,PTPRC_0_146,.,+,chr1:198696858-198696908(+),CCCAAGTATCCCCGGACTCTTTGGATAATGCTAGTGCTTTTAATAC...,TGTGGTATTAAAAGCACTAGCATTATCCAAAGAGTCCGGGGATACT...,0.44,4,0.32,0.56


In [27]:
## Check for required nucleotides in specific positions:
if required_nts is not None:
    kmers['has_required_nts'] = check_for_required_nts(kmers, required_nts)
    print(kmers['has_required_nts'].value_counts())
    ## Filter for required nucleotides
    kmers = kmers[kmers['has_required_nts']==True].reset_index(drop=True)

has_required_nts
False    106
True      42
Name: count, dtype: int64


In [28]:
## Export kmers before filtering
kmers.to_csv((out_dir + 'kmers_candidates_unfiltered.csv'))

In [29]:
kmers

,seqname,start,end,name,score,strand,kmer_coord,transcript_seq,probe_seq,GC_content_full,longest_homopolymer,GC_content_LHS,GC_content_RHS,has_required_nts
0,chr1,198696717,198696767,PTPRC_0_5,.,+,chr1:198696717-198696767(+),CTACAGCAAAGATGCCCAGTGTTCCACTTTCAAGTGACCCCTTACC...,AGTAGGTAAGGGGTCACTTGAAAGTGGAACACTGGGCATCTTTGCT...,0.48,4,0.44,0.52,True
1,chr1,198696723,198696773,PTPRC_0_11,.,+,chr1:198696723-198696773(+),CAAAGATGCCCAGTGTTCCACTTTCAAGTGACCCCTTACCTACTCA...,GGTGTGAGTAGGTAAGGGGTCACTTGAAAGTGGAACACTGGGCATC...,0.50,4,0.52,0.48,True
2,chr1,198696724,198696774,PTPRC_0_12,.,+,chr1:198696724-198696774(+),AAAGATGCCCAGTGTTCCACTTTCAAGTGACCCCTTACCTACTCAC...,TGGTGTGAGTAGGTAAGGGGTCACTTGAAAGTGGAACACTGGGCAT...,0.48,4,0.52,0.44,True
3,chr1,198696728,198696778,PTPRC_0_16,.,+,chr1:198696728-198696778(+),ATGCCCAGTGTTCCACTTTCAAGTGACCCCTTACCTACTCACACCA...,GCAGTGGTGTGAGTAGGTAAGGGGTCACTTGAAAGTGGAACACTGG...,0.52,4,0.56,0.48,True
4,chr1,198696735,198696785,PTPRC_0_23,.,+,chr1:198696735-198696785(+),GTGTTCCACTTTCAAGTGACCCCTTACCTACTCACACCACTGCATT...,TGAGAATGCAGTGGTGTGAGTAGGTAAGGGGTCACTTGAAAGTGGA...,0.48,4,0.48,0.48,True
5,chr1,198696739,198696789,PTPRC_0_27,.,+,chr1:198696739-198696789(+),TCCACTTTCAAGTGACCCCTTACCTACTCACACCACTGCATTCTCA...,CGGGTGAGAATGCAGTGGTGTGAGTAGGTAAGGGGTCACTTGAAAG...,0.52,4,0.56,0.48,True
6,chr1,198696743,198696793,PTPRC_0_31,.,+,chr1:198696743-198696793(+),CTTTCAAGTGACCCCTTACCTACTCACACCACTGCATTCTCACCCG...,CTTGCGGGTGAGAATGCAGTGGTGTGAGTAGGTAAGGGGTCACTTG...,0.52,4,0.56,0.48,True
7,chr1,198696745,198696795,PTPRC_0_33,.,+,chr1:198696745-198696795(+),TTCAAGTGACCCCTTACCTACTCACACCACTGCATTCTCACCCGCA...,TGCTTGCGGGTGAGAATGCAGTGGTGTGAGTAGGTAAGGGGTCACT...,0.52,4,0.56,0.48,True
8,chr1,198696748,198696798,PTPRC_0_36,.,+,chr1:198696748-198696798(+),AAGTGACCCCTTACCTACTCACACCACTGCATTCTCACCCGCAAGC...,AGGTGCTTGCGGGTGAGAATGCAGTGGTGTGAGTAGGTAAGGGGTC...,0.54,4,0.56,0.52,True
9,chr1,198696753,198696803,PTPRC_0_41,.,+,chr1:198696753-198696803(+),ACCCCTTACCTACTCACACCACTGCATTCTCACCCGCAAGCACCTT...,TTCAAAGGTGCTTGCGGGTGAGAATGCAGTGGTGTGAGTAGGTAAG...,0.52,4,0.48,0.56,True


In [30]:
## Filter for GC content
kmers = filter_by_GC_content(kmers, min_GC, max_GC)

In [31]:
kmers

,seqname,start,end,name,score,strand,kmer_coord,transcript_seq,probe_seq,GC_content_full,longest_homopolymer,GC_content_LHS,GC_content_RHS,has_required_nts
0,chr1,198696723,198696773,PTPRC_0_11,.,+,chr1:198696723-198696773(+),CAAAGATGCCCAGTGTTCCACTTTCAAGTGACCCCTTACCTACTCA...,GGTGTGAGTAGGTAAGGGGTCACTTGAAAGTGGAACACTGGGCATC...,0.50,4,0.52,0.48,True
1,chr1,198696728,198696778,PTPRC_0_16,.,+,chr1:198696728-198696778(+),ATGCCCAGTGTTCCACTTTCAAGTGACCCCTTACCTACTCACACCA...,GCAGTGGTGTGAGTAGGTAAGGGGTCACTTGAAAGTGGAACACTGG...,0.52,4,0.56,0.48,True
2,chr1,198696735,198696785,PTPRC_0_23,.,+,chr1:198696735-198696785(+),GTGTTCCACTTTCAAGTGACCCCTTACCTACTCACACCACTGCATT...,TGAGAATGCAGTGGTGTGAGTAGGTAAGGGGTCACTTGAAAGTGGA...,0.48,4,0.48,0.48,True
3,chr1,198696739,198696789,PTPRC_0_27,.,+,chr1:198696739-198696789(+),TCCACTTTCAAGTGACCCCTTACCTACTCACACCACTGCATTCTCA...,CGGGTGAGAATGCAGTGGTGTGAGTAGGTAAGGGGTCACTTGAAAG...,0.52,4,0.56,0.48,True
4,chr1,198696743,198696793,PTPRC_0_31,.,+,chr1:198696743-198696793(+),CTTTCAAGTGACCCCTTACCTACTCACACCACTGCATTCTCACCCG...,CTTGCGGGTGAGAATGCAGTGGTGTGAGTAGGTAAGGGGTCACTTG...,0.52,4,0.56,0.48,True
5,chr1,198696745,198696795,PTPRC_0_33,.,+,chr1:198696745-198696795(+),TTCAAGTGACCCCTTACCTACTCACACCACTGCATTCTCACCCGCA...,TGCTTGCGGGTGAGAATGCAGTGGTGTGAGTAGGTAAGGGGTCACT...,0.52,4,0.56,0.48,True
6,chr1,198696748,198696798,PTPRC_0_36,.,+,chr1:198696748-198696798(+),AAGTGACCCCTTACCTACTCACACCACTGCATTCTCACCCGCAAGC...,AGGTGCTTGCGGGTGAGAATGCAGTGGTGTGAGTAGGTAAGGGGTC...,0.54,4,0.56,0.52,True
7,chr1,198696753,198696803,PTPRC_0_41,.,+,chr1:198696753-198696803(+),ACCCCTTACCTACTCACACCACTGCATTCTCACCCGCAAGCACCTT...,TTCAAAGGTGCTTGCGGGTGAGAATGCAGTGGTGTGAGTAGGTAAG...,0.52,4,0.48,0.56,True
8,chr1,198696759,198696809,PTPRC_0_47,.,+,chr1:198696759-198696809(+),TACCTACTCACACCACTGCATTCTCACCCGCAAGCACCTTTGAAAG...,TTCTCTTTCAAAGGTGCTTGCGGGTGAGAATGCAGTGGTGTGAGTA...,0.48,3,0.48,0.48,True
9,chr1,198696821,198696871,PTPRC_0_109,.,+,chr1:198696821-198696871(+),GAGACCACAACTTCTCTTAGTCCAGACAATACTTCCACCCAAGTAT...,GGGGATACTTGGGTGGAAGTATTGTCTGGACTAAGAGAAGTTGTGG...,0.48,4,0.48,0.48,True


In [32]:
## Candidate kmers
kmers

,seqname,start,end,name,score,strand,kmer_coord,transcript_seq,probe_seq,GC_content_full,longest_homopolymer,GC_content_LHS,GC_content_RHS,has_required_nts
0,chr1,198696723,198696773,PTPRC_0_11,.,+,chr1:198696723-198696773(+),CAAAGATGCCCAGTGTTCCACTTTCAAGTGACCCCTTACCTACTCA...,GGTGTGAGTAGGTAAGGGGTCACTTGAAAGTGGAACACTGGGCATC...,0.50,4,0.52,0.48,True
1,chr1,198696728,198696778,PTPRC_0_16,.,+,chr1:198696728-198696778(+),ATGCCCAGTGTTCCACTTTCAAGTGACCCCTTACCTACTCACACCA...,GCAGTGGTGTGAGTAGGTAAGGGGTCACTTGAAAGTGGAACACTGG...,0.52,4,0.56,0.48,True
2,chr1,198696735,198696785,PTPRC_0_23,.,+,chr1:198696735-198696785(+),GTGTTCCACTTTCAAGTGACCCCTTACCTACTCACACCACTGCATT...,TGAGAATGCAGTGGTGTGAGTAGGTAAGGGGTCACTTGAAAGTGGA...,0.48,4,0.48,0.48,True
3,chr1,198696739,198696789,PTPRC_0_27,.,+,chr1:198696739-198696789(+),TCCACTTTCAAGTGACCCCTTACCTACTCACACCACTGCATTCTCA...,CGGGTGAGAATGCAGTGGTGTGAGTAGGTAAGGGGTCACTTGAAAG...,0.52,4,0.56,0.48,True
4,chr1,198696743,198696793,PTPRC_0_31,.,+,chr1:198696743-198696793(+),CTTTCAAGTGACCCCTTACCTACTCACACCACTGCATTCTCACCCG...,CTTGCGGGTGAGAATGCAGTGGTGTGAGTAGGTAAGGGGTCACTTG...,0.52,4,0.56,0.48,True
5,chr1,198696745,198696795,PTPRC_0_33,.,+,chr1:198696745-198696795(+),TTCAAGTGACCCCTTACCTACTCACACCACTGCATTCTCACCCGCA...,TGCTTGCGGGTGAGAATGCAGTGGTGTGAGTAGGTAAGGGGTCACT...,0.52,4,0.56,0.48,True
6,chr1,198696748,198696798,PTPRC_0_36,.,+,chr1:198696748-198696798(+),AAGTGACCCCTTACCTACTCACACCACTGCATTCTCACCCGCAAGC...,AGGTGCTTGCGGGTGAGAATGCAGTGGTGTGAGTAGGTAAGGGGTC...,0.54,4,0.56,0.52,True
7,chr1,198696753,198696803,PTPRC_0_41,.,+,chr1:198696753-198696803(+),ACCCCTTACCTACTCACACCACTGCATTCTCACCCGCAAGCACCTT...,TTCAAAGGTGCTTGCGGGTGAGAATGCAGTGGTGTGAGTAGGTAAG...,0.52,4,0.48,0.56,True
8,chr1,198696759,198696809,PTPRC_0_47,.,+,chr1:198696759-198696809(+),TACCTACTCACACCACTGCATTCTCACCCGCAAGCACCTTTGAAAG...,TTCTCTTTCAAAGGTGCTTGCGGGTGAGAATGCAGTGGTGTGAGTA...,0.48,3,0.48,0.48,True
9,chr1,198696821,198696871,PTPRC_0_109,.,+,chr1:198696821-198696871(+),GAGACCACAACTTCTCTTAGTCCAGACAATACTTCCACCCAAGTAT...,GGGGATACTTGGGTGGAAGTATTGTCTGGACTAAGAGAAGTTGTGG...,0.48,4,0.48,0.48,True


In [33]:
kmers.to_csv((out_dir + 'kmers_candidates_filtered.csv'))

### 5. Remove probes with potential off-targets

Having identified a set of kmers that fulfill our sequence requirements, we can next proceed with testing whether they are specific to our transcript/exon of interest.

For this we rely on using BLAST. 

We recommend blasting against transcripts (i.e., exons and introns combined) to be as conservative as possible in terms of off-targets. 
However, in cases where it is not possible to obtain enough suitable kmers (e.g., for  short transcripts), it is reasonable to relax this requirement by BLASTing against exons only (a much smaller search space).

At this step, we also want to consider whether our probes are split (as in the current specifications for VisiumHD) or a single oligo. If probes are split, it's best to BLAST each side separately, to make sure that both sides are specific.

In [34]:
## The first thing to do is to export our sequences in fasta format, so that we can use them for BLAST
write_fasta(kmers['name'], kmers['transcript_seq'], (out_dir + 'kmers_candidates_filtered_transcript_seqs.fa'))
## If our probes are meant to be split, we should additionally blast them separately 
## Note that the LHS/RHS in the transcript are reversed compared to the probe (i.e., the LHS of the transcript is complementary to the RHS of the probe)
if split_nt is not None: 
    ## Make split probes
    kmers['transcript_seq_LHS'] = [seq[0:split_nt] for seq in kmers['transcript_seq']]
    kmers['transcript_seq_RHS'] = [seq[split_nt: probe_length] for seq in kmers['transcript_seq']]

    ## We are exporting the transcript sequence as that's the one that has to be blasted against the human transcriptome
    write_fasta(kmers['name'], kmers['transcript_seq_LHS'], (out_dir + 'kmers_candidates_filtered_transcript_seqs_LHS.fa'))
    write_fasta(kmers['name'], kmers['transcript_seq_RHS'], (out_dir + 'kmers_candidates_filtered_transcript_seqs_RHS.fa'))    

In [35]:
blast_res = {}
## First, blast the full probe
blast_res['full'] = run_blast(fasta=(out_dir + 'kmers_candidates_filtered_transcript_seqs.fa'),
                              blastdb = blast_db,
                              path2blastn=(blast_exec_path + 'blastn'),
                              outfile = (out_dir + 'kmers_candidates_filtered_blast_output.txt'))

## Additionally, if probe is split, blast each side separately
if split_nt is not None: 
    blast_res['LHS'] = run_blast(fasta=(out_dir + 'kmers_candidates_filtered_transcript_seqs_LHS.fa'),
                                     blastdb = blast_db,
                                     path2blastn=(blast_exec_path + 'blastn'),
                                     outfile = (out_dir + 'kmers_candidates_filtered_blast_output_LHS.txt'))
    blast_res['RHS'] = run_blast(fasta=(out_dir + 'kmers_candidates_filtered_transcript_seqs_RHS.fa'),
                                     blastdb = blast_db,
                                     path2blastn=(blast_exec_path + 'blastn'),
                                     outfile = (out_dir + 'kmers_candidates_filtered_blast_output_RHS.txt'))

In [36]:
for k in blast_res.keys():
    print(("The following genes were detected in mode: " +  k))
    print(blast_res[k]['sgeneid'].value_counts().head(10))

The following genes were detected in mode: full
sgeneid
PTPRC    132
Name: count, dtype: int64
The following genes were detected in mode: LHS
sgeneid
PTPRC       132
PDE4DIP      68
RIN2         20
C12orf56      7
MMP16         2
Name: count, dtype: int64
The following genes were detected in mode: RHS
sgeneid
PTPRC           132
KANSL1L          34
MAGI2            28
C12orf56          7
LOC107985977      2
SEPTIN7P15        1
SEPTIN7P5         1
Name: count, dtype: int64


In [37]:
blast_res['full']

,name,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore,sgeneid
0,PTPRC_0_11,PTPRC::chr1:198638712-198757476(+),100.0,50,0,0,1,50,58012,58061,8.510000e-17,91.5,PTPRC
1,PTPRC_0_11,PTPRC::chr1:198638712-198757476(+),100.0,50,0,0,1,50,58012,58061,8.510000e-17,91.5,PTPRC
2,PTPRC_0_11,PTPRC::chr1:198638712-198757476(+),100.0,50,0,0,1,50,58012,58061,8.510000e-17,91.5,PTPRC
3,PTPRC_0_11,PTPRC::chr1:198638712-198757476(+),100.0,50,0,0,1,50,58012,58061,8.510000e-17,91.5,PTPRC
4,PTPRC_0_11,PTPRC::chr1:198639039-198757476(+),100.0,50,0,0,1,50,57685,57734,8.510000e-17,91.5,PTPRC
...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,PTPRC_0_128,PTPRC::chr1:198639039-198757476(+),100.0,50,0,0,1,50,57802,57851,8.510000e-17,91.5,PTPRC
128,PTPRC_0_128,PTPRC::chr1:198639039-198757476(+),100.0,50,0,0,1,50,57802,57851,8.510000e-17,91.5,PTPRC
129,PTPRC_0_128,PTPRC::chr1:198639039-198757476(+),100.0,50,0,0,1,50,57802,57851,8.510000e-17,91.5,PTPRC
130,PTPRC_0_128,PTPRC::chr1_GL383520v2_alt:280444-359708(+),100.0,50,0,0,1,50,57802,57851,8.510000e-17,91.5,PTPRC


Not all BLAST hits will be off-targets. Hopefully, our gene of interest is included in the BLAST output. We therefore need to filter for hits with different gene IDs.

In [38]:
offtargets = []
for k in blast_res.keys():
    offtargets += (detect_offtargets(blast_res[k], gene_ID, min_mismatches=min_mismatches))
## Remove redundancies
offtargets = list(set(offtargets))

In [39]:
len(offtargets)

6

In [40]:
## Remove off-targets
kmers = kmers[kmers['name'].isin(offtargets)==False].reset_index(drop=True)

In [41]:
kmers 

,seqname,start,end,name,score,strand,kmer_coord,transcript_seq,probe_seq,GC_content_full,longest_homopolymer,GC_content_LHS,GC_content_RHS,has_required_nts,transcript_seq_LHS,transcript_seq_RHS
0,chr1,198696745,198696795,PTPRC_0_33,.,+,chr1:198696745-198696795(+),TTCAAGTGACCCCTTACCTACTCACACCACTGCATTCTCACCCGCA...,TGCTTGCGGGTGAGAATGCAGTGGTGTGAGTAGGTAAGGGGTCACT...,0.52,4,0.56,0.48,True,TTCAAGTGACCCCTTACCTACTCAC,ACCACTGCATTCTCACCCGCAAGCA
1,chr1,198696748,198696798,PTPRC_0_36,.,+,chr1:198696748-198696798(+),AAGTGACCCCTTACCTACTCACACCACTGCATTCTCACCCGCAAGC...,AGGTGCTTGCGGGTGAGAATGCAGTGGTGTGAGTAGGTAAGGGGTC...,0.54,4,0.56,0.52,True,AAGTGACCCCTTACCTACTCACACC,ACTGCATTCTCACCCGCAAGCACCT
2,chr1,198696753,198696803,PTPRC_0_41,.,+,chr1:198696753-198696803(+),ACCCCTTACCTACTCACACCACTGCATTCTCACCCGCAAGCACCTT...,TTCAAAGGTGCTTGCGGGTGAGAATGCAGTGGTGTGAGTAGGTAAG...,0.52,4,0.48,0.56,True,ACCCCTTACCTACTCACACCACTGC,ATTCTCACCCGCAAGCACCTTTGAA
3,chr1,198696821,198696871,PTPRC_0_109,.,+,chr1:198696821-198696871(+),GAGACCACAACTTCTCTTAGTCCAGACAATACTTCCACCCAAGTAT...,GGGGATACTTGGGTGGAAGTATTGTCTGGACTAAGAGAAGTTGTGG...,0.48,4,0.48,0.48,True,GAGACCACAACTTCTCTTAGTCCAG,ACAATACTTCCACCCAAGTATCCCC
4,chr1,198696823,198696873,PTPRC_0_111,.,+,chr1:198696823-198696873(+),GACCACAACTTCTCTTAGTCCAGACAATACTTCCACCCAAGTATCC...,CCGGGGATACTTGGGTGGAAGTATTGTCTGGACTAAGAGAAGTTGT...,0.50,4,0.52,0.48,True,GACCACAACTTCTCTTAGTCCAGAC,AATACTTCCACCCAAGTATCCCCGG
5,chr1,198696840,198696890,PTPRC_0_128,.,+,chr1:198696840-198696890(+),GTCCAGACAATACTTCCACCCAAGTATCCCCGGACTCTTTGGATAA...,AGCATTATCCAAAGAGTCCGGGGATACTTGGGTGGAAGTATTGTCT...,0.48,4,0.48,0.48,True,GTCCAGACAATACTTCCACCCAAGT,ATCCCCGGACTCTTTGGATAATGCT


### 6. Select a probe

At this point, we only have 5 potential probes, but we only need one. Since they all have the same homopolymer length (4), we can pick the one that is most downstream, to  have the maximum distance from the existing probes, one of which partially captures exon 3.

In [45]:
## Sort in increasing homopolymer length
kmers = kmers.sort_values('end', ascending=False).reset_index(drop=True)

In [46]:
kmers

,seqname,start,end,name,score,strand,kmer_coord,transcript_seq,probe_seq,GC_content_full,longest_homopolymer,GC_content_LHS,GC_content_RHS,has_required_nts,transcript_seq_LHS,transcript_seq_RHS
0,chr1,198696840,198696890,PTPRC_0_128,.,+,chr1:198696840-198696890(+),GTCCAGACAATACTTCCACCCAAGTATCCCCGGACTCTTTGGATAA...,AGCATTATCCAAAGAGTCCGGGGATACTTGGGTGGAAGTATTGTCT...,0.48,4,0.48,0.48,True,GTCCAGACAATACTTCCACCCAAGT,ATCCCCGGACTCTTTGGATAATGCT
1,chr1,198696823,198696873,PTPRC_0_111,.,+,chr1:198696823-198696873(+),GACCACAACTTCTCTTAGTCCAGACAATACTTCCACCCAAGTATCC...,CCGGGGATACTTGGGTGGAAGTATTGTCTGGACTAAGAGAAGTTGT...,0.50,4,0.52,0.48,True,GACCACAACTTCTCTTAGTCCAGAC,AATACTTCCACCCAAGTATCCCCGG
2,chr1,198696821,198696871,PTPRC_0_109,.,+,chr1:198696821-198696871(+),GAGACCACAACTTCTCTTAGTCCAGACAATACTTCCACCCAAGTAT...,GGGGATACTTGGGTGGAAGTATTGTCTGGACTAAGAGAAGTTGTGG...,0.48,4,0.48,0.48,True,GAGACCACAACTTCTCTTAGTCCAG,ACAATACTTCCACCCAAGTATCCCC
3,chr1,198696753,198696803,PTPRC_0_41,.,+,chr1:198696753-198696803(+),ACCCCTTACCTACTCACACCACTGCATTCTCACCCGCAAGCACCTT...,TTCAAAGGTGCTTGCGGGTGAGAATGCAGTGGTGTGAGTAGGTAAG...,0.52,4,0.48,0.56,True,ACCCCTTACCTACTCACACCACTGC,ATTCTCACCCGCAAGCACCTTTGAA
4,chr1,198696748,198696798,PTPRC_0_36,.,+,chr1:198696748-198696798(+),AAGTGACCCCTTACCTACTCACACCACTGCATTCTCACCCGCAAGC...,AGGTGCTTGCGGGTGAGAATGCAGTGGTGTGAGTAGGTAAGGGGTC...,0.54,4,0.56,0.52,True,AAGTGACCCCTTACCTACTCACACC,ACTGCATTCTCACCCGCAAGCACCT
5,chr1,198696745,198696795,PTPRC_0_33,.,+,chr1:198696745-198696795(+),TTCAAGTGACCCCTTACCTACTCACACCACTGCATTCTCACCCGCA...,TGCTTGCGGGTGAGAATGCAGTGGTGTGAGTAGGTAAGGGGTCACT...,0.52,4,0.56,0.48,True,TTCAAGTGACCCCTTACCTACTCAC,ACCACTGCATTCTCACCCGCAAGCA


In [47]:
selected_probes_list = []
df = kmers.copy()
i = 0
while i < n_desired_probes:
    selected_probes_list.append(df.iloc[0:1,:])
    ## Going for a high offset as we have a large span:
    df = remove_overlapping_probes(df,
                                   df['name'][0], 
                                   offset=probe_offset).reset_index(drop=True).copy()
    i +=1

/nfs/team205/is10/miniconda/envs/gene2probe_env/lib/python3.11/site-packages/gene2probe/gene2probe.py:436: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  start = int(probe_df['start'].iloc[i])
/nfs/team205/is10/miniconda/envs/gene2probe_env/lib/python3.11/site-packages/gene2probe/gene2probe.py:437: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  end = int(probe_df['end'].iloc[i])


In [48]:
selected_probes_df = pd.concat(selected_probes_list, axis=0).reset_index(drop=True)

In [50]:
selected_probes_df

,seqname,start,end,name,score,strand,kmer_coord,transcript_seq,probe_seq,GC_content_full,longest_homopolymer,GC_content_LHS,GC_content_RHS,has_required_nts,transcript_seq_LHS,transcript_seq_RHS
0,chr1,198696840,198696890,PTPRC_0_128,.,+,chr1:198696840-198696890(+),GTCCAGACAATACTTCCACCCAAGTATCCCCGGACTCTTTGGATAA...,AGCATTATCCAAAGAGTCCGGGGATACTTGGGTGGAAGTATTGTCT...,0.48,4,0.48,0.48,True,GTCCAGACAATACTTCCACCCAAGT,ATCCCCGGACTCTTTGGATAATGCT


In [51]:
for seq in selected_probes_df['transcript_seq']:
    print(seq)

GTCCAGACAATACTTCCACCCAAGTATCCCCGGACTCTTTGGATAATGCT


And we are done! We have now selected a probe that could help us detect if exon 4 of PTPRC is being used.

Since our probes are meant to be split, we can additionally generate these columns:

In [52]:
if split_nt is not None: 
    ## Make split probes (and add adapters if provided)
    selected_probes_df['probe_seq_LHS'] = [(LHS_pref + seq[0:split_nt] + LHS_suff) for seq in selected_probes_df['probe_seq']]
    selected_probes_df['probe_seq_RHS'] = [(RHS_pref +seq[split_nt: probe_length] + RHS_suff) for seq in selected_probes_df['probe_seq']]

In [53]:
## Also add gene_ID for completeness
selected_probes_df['gene_ID'] = gene_ID

In [54]:
## Export selected probes as dataframe:
selected_probes_df.to_csv((out_dir + 'kmers_selected_probes.csv'))

We always recommend additionally performing a manual BLAST of these probe sequences to make sure that there are no off-target effects.

In [55]:
selected_probes_df

,seqname,start,end,name,score,strand,kmer_coord,transcript_seq,probe_seq,GC_content_full,longest_homopolymer,GC_content_LHS,GC_content_RHS,has_required_nts,transcript_seq_LHS,transcript_seq_RHS,probe_seq_LHS,probe_seq_RHS,gene_ID
0,chr1,198696840,198696890,PTPRC_0_128,.,+,chr1:198696840-198696890(+),GTCCAGACAATACTTCCACCCAAGTATCCCCGGACTCTTTGGATAA...,AGCATTATCCAAAGAGTCCGGGGATACTTGGGTGGAAGTATTGTCT...,0.48,4,0.48,0.48,True,GTCCAGACAATACTTCCACCCAAGT,ATCCCCGGACTCTTTGGATAATGCT,CCTTGGCACCCGAGAATTCCAAGCATTATCCAAAGAGTCCGGGGAT,/5Phos/ACTTGGGTGGAAGTATTGTCTGGACAAAAAAAAAAAAAA...,PTPRC
